In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,247 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/dead

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
file_path = SparkFiles.get("home_sales_revised.csv")

df = spark.read.csv(file_path, header=True, inferSchema=True)

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df = "home_sales_view"
df.createOrReplaceTempView(home_sales_df)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
SELECT YEAR(date) AS sale_year, AVG(price) AS avg_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY sale_year
ORDER BY sale_year
"""

# Run the SQL query and store the result in a DataFrame
four_bed = spark.sql(query)

# Show the result
four_bed.show()

+---------+------------------+
|sale_year|         avg_price|
+---------+------------------+
|     2019| 300263.6955128205|
|     2020|298353.78003169573|
|     2021|   301819.44398864|
|     2022| 296363.8845050215|
+---------+------------------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built AS build_year, AVG(price) AS avg_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY build_year
ORDER BY build_year
"""

# Run the SQL query and store the result in a DataFrame
three_bed_three_bath = spark.sql(query)

# Show the result
three_bed_three_bath.show()


+----------+------------------+
|build_year|         avg_price|
+----------+------------------+
|      2010|  292859.615942029|
|      2011|291117.46706586826|
|      2012| 293683.1872074883|
|      2013|295962.27145085804|
|      2014| 290852.2661870504|
|      2015| 288770.2966101695|
|      2016|  290555.073964497|
|      2017| 292676.7887740029|
+----------+------------------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built AS build_year, AVG(price) AS avg_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY build_year
ORDER BY build_year
"""

# Run the SQL query and store the result in a DataFrame
avg_price = spark.sql(query)

# Show the result
avg_price.show()


+----------+------------------+
|build_year|         avg_price|
+----------+------------------+
|      2010| 285010.2215909091|
|      2011| 276553.8128654971|
|      2012|307539.97402597405|
|      2013|      303676.79375|
|      2014| 298264.7183908046|
|      2015| 297609.9679144385|
|      2016| 293965.1046511628|
|      2017|280317.57692307694|
+----------+------------------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT AVG(view) AS avg_view
FROM home_sales_view
WHERE price >= 350000
"""

# Run the SQL query and store the result in a DataFrame
view_rating = spark.sql(query)

# Show the result
view_rating.show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|  32.264|
+--------+

--- 0.9640235900878906 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Start timing for uncached query
uncached_start_time = time.time()

# Run a SQL query to calculate the average "view" rating for homes with price >= $350,000
query = """
SELECT AVG(view) AS avg_view
FROM home_sales_view
WHERE price >= 350000
"""

# Run the SQL query
uncached_result_df = spark.sql(query)

# Calculate and print the uncached run time
uncached_run_time = time.time() - uncached_start_time
print("Uncached query runtime: %.4f seconds" % uncached_run_time)

# Cache the temporary table
df.cache()

# Start timing for cached query
cached_start_time = time.time()

# Run the same SQL query using the cached data
cached_result_df = spark.sql(query)

# Calculate and print the cached run time
cached_run_time = time.time() - cached_start_time
print("Cached query runtime: %.4f seconds" % cached_run_time)



Uncached query runtime: 0.0381 seconds
Cached query runtime: 0.0602 seconds


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Define the output Parquet path with partitioning
output_path = "path_to_output_directory"

# Write the DataFrame to Parquet with partitioning by "date_built"
df.write.partitionBy("date_built").parquet(output_path)


In [20]:
# 11. Read the parquet formatted data.
# Define the path to the Parquet data
parquet_path = output_path

# Read the Parquet data
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame
parquet_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
# Start timing for Parquet query
parquet_start_time = time.time()

# Run a SQL query to calculate the average "view" rating for homes with price >= $350,000
query = """
SELECT AVG(view) AS avg_view
FROM parquet_home_sales
WHERE price >= 350000
"""

# Run the SQL query on the Parquet DataFrame
parquet_result_df = spark.sql(query)

# Calculate and print the Parquet query runtime
parquet_run_time = time.time() - parquet_start_time
print("Parquet query runtime: %.4f seconds" % parquet_run_time)

# Start timing for cached query (assumes you have already cached the DataFrame)
cached_start_time = time.time()

# Run the same SQL query using the cached DataFrame
cached_result_df = spark.sql(query)

# Calculate and print the cached query runtime
cached_run_time = time.time() - cached_start_time
print("Cached query runtime: %.4f seconds" % cached_run_time)


Parquet query runtime: 0.0341 seconds
Cached query runtime: 0.0169 seconds


In [25]:
# 14. Uncache the home_sales temporary table.
df.unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [26]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales_view')

# Print the result
print("Is home_sales_view cached:", is_cached)


Is home_sales_view cached: False
